# [BiographySampo](http://biografiasampo.fi/haku/ruudukko) artworks from Wikidata

Install the dependencies:

In [1]:
!pip install SPARQLWrapper numpy scipy matplotlib pandas networkx
%matplotlib inline

import codecs
import datetime
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import numpy    as np
import networkx as nx
import pandas   as pd
import re
import seaborn as sns
import sys

from collections import Counter, OrderedDict, defaultdict
from itertools import product
from rdflib.namespace import XSD, Namespace
from rdflib import URIRef
from statistics import mean
from scipy.stats import binned_statistic
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from scipy.signal import convolve2d

from SPARQLWrapper import SPARQLWrapper, JSON, POST

import logging
LOGGER = logging.getLogger(sys.argv[0])


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Define some useful functions for data convertion

In [2]:
DATATYPECONVERTERS = {
      str(XSD.integer):  int,
      # str(XSD.date):     lambda v: datetime.datetime.strptime(v, '%Y-%m-%d').date()
      str(XSD.decimal):  float,
      # XSD.double
  }

def convertDatatype(obj):
  return DATATYPECONVERTERS.get(obj.get('datatype'), str)(obj.get('value')) 

def convertDatatypes(results):
    res = results["results"]["bindings"]
    return [dict([(k, convertDatatype(v)) for k,v in r.items()]) for r in res]

def JSON2Pandas(results):
    return pd.DataFrame(convertDatatypes(results))

from rdflib import Namespace, URIRef, Graph, Literal, XSD
from rdflib.namespace import RDF, SKOS, OWL, FOAF, RDFS, XSD, DCTERMS
FOAF = Namespace(FOAF)

BIOC = Namespace("http://ldf.fi/schema/bioc/")
CRM = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
GVP = Namespace("http://vocab.getty.edu/ontology#")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
LABEL = Namespace("http://ldf.fi/nbf/label/")
SCHEMA = Namespace("http://schema.org/")
SKOSXL = Namespace("http://www.w3.org/2008/05/skos-xl#")

NBF = Namespace("http://ldf.fi/nbf/")
IDM = Namespace("http://www.intavia.eu/")
IDMCORE = Namespace("http://www.intavia.eu/idm-core/")

IDM_PERSON = Namespace('http://www.intavia.eu/personproxy/bs/')
IDM_PROVIDED_PERSON = Namespace('http://www.intavia.eu/provided_person/bs/')
IDM_PLACE = Namespace('http://www.intavia.eu/placeproxy/bs/')

TIMES = Namespace("http://ldf.fi/nbf/times/")
TITLES = Namespace("http://ldf.fi/nbf/titles/")
PLACES = Namespace("http://ldf.fi/nbf/places/")
SOURCES = Namespace("http://ldf.fi/nbf/sources/")

WD = Namespace("http://www.wikidata.org/entity/")

PREFIXES = """PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX bioc: <http://ldf.fi/schema/bioc/>
PREFIX skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX nbf: <http://ldf.fi/nbf/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX schema: <http://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#> 
PREFIX sources: <http://ldf.fi/nbf/sources/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
PREFIX idm-core: <http://www.intavia.eu/idm-core/>
"""

def initGraph(g = None):
    
    if not g:
        g=Graph()
    
    # bind prefixes
    g.bind("dct", DCTERMS)
    g.bind("gvp", GVP)
    g.bind("owl", OWL)
    g.bind("foaf", FOAF)
    g.bind("idm", IDM)
    g.bind("idm-core", IDMCORE)
    g.bind("rdf", RDF)
    g.bind("rdfs", RDFS)
    g.bind("schema", SCHEMA)
    g.bind("sources", SOURCES)
    g.bind("skos",SKOS)
    g.bind("skosxl", SKOSXL)
    g.bind("xsd", XSD)
    g.bind("geo", GEO)
    g.bind("crm", CRM)
    g.bind("idnametype", Namespace('http://www.intavia.eu/nametype/'))
    
    g.bind("bioc", BIOC)
    g.bind("label", LABEL)
    g.bind("nbf", NBF)
    g.bind("times", TIMES)
    g.bind("wd", WD)
    
    return g

def saveGraphs(tuples, silent=False):
    
    for g, outfile in tuples:
        
        if len(g):
            
            args = sys.argv
            
            outf = codecs.open(outfile, encoding='utf-8', mode='w')
            
            outf.write("# Created with script {}\n".format(args[0]))
            outf.write("# in colab https://colab.research.google.com/drive/1c6hwZs1ooi1G1kfHeB-JYeywemyY-mU-?usp=sharing")
            outf.write("# python3 {}\n".format(' '.join(args)))
            
            now = datetime.datetime.now()
            outf.write("# {}\n\n".format(now.strftime("%Y-%m-%d %H:%M")))
            
            # g.serialize(destination=outfile, format='turtle')
            #  outf.write( g.serialize(format='turtle').decode("utf-8") )
            outf.write( g.serialize(format='turtle') )
            
            outf.close()
            
            if not silent:
                LOGGER.info("{} triples saved to {}".format(len(g),outfile))
            
        elif not silent:
            LOGGER.info("No output to {}".format(outfile))

# Quering NBF people data
[yasgui.com](https://api.triplydb.com/s/fosA2MYMf)

In [3]:
sparql = SPARQLWrapper("https://ldf.fi/nbf/sparql")
# https://api.triplydb.com/s/fosA2MYMf
sparql.setQuery(PREFIXES +
""" SELECT DISTINCT ?id ?wiki 
WHERE {
  VALUES ?source { sources:source1 }
  ?id a nbf:PersonConcept ;
   dct:source ?source ;
   foaf:focus/schema:gender ?gender ;
   nbf:wikidata ?wiki .
} """)

sparql.setReturnFormat(JSON)
sparql.setMethod('POST')
# sparql.addCustomHttpHeader()
results = sparql.query().convert()

res = results["results"]["bindings"]
print(f"{len(res)} results")

print(f"Datafields {results['head']['vars']}")
res = convertDatatypes(results)

nbf2wiki = res

for ob in res[:10]:
  print(ob)

6029 results
Datafields ['id', 'wiki']
{'id': 'http://ldf.fi/nbf/p1869', 'wiki': 'http://www.wikidata.org/entity/Q4063683'}
{'id': 'http://ldf.fi/nbf/p3876', 'wiki': 'http://www.wikidata.org/entity/Q11865392'}
{'id': 'http://ldf.fi/nbf/p4912', 'wiki': 'http://www.wikidata.org/entity/Q3268888'}
{'id': 'http://ldf.fi/nbf/p6241', 'wiki': 'http://www.wikidata.org/entity/Q18760019'}
{'id': 'http://ldf.fi/nbf/p249', 'wiki': 'http://www.wikidata.org/entity/Q6224449'}
{'id': 'http://ldf.fi/nbf/p2678', 'wiki': 'http://www.wikidata.org/entity/Q11886278'}
{'id': 'http://ldf.fi/nbf/p2415', 'wiki': 'http://www.wikidata.org/entity/Q11886018'}
{'id': 'http://ldf.fi/nbf/p1880', 'wiki': 'http://www.wikidata.org/entity/Q5561382'}
{'id': 'http://ldf.fi/nbf/p282', 'wiki': 'http://www.wikidata.org/entity/Q233277'}
{'id': 'http://ldf.fi/nbf/p2546', 'wiki': 'http://www.wikidata.org/entity/Q297032'}


In [4]:
# Europeana endpoint rejects too long queries, so to shorten apply prefices
def applyPrefix(st):
  st = re.sub(r'http://www.wikidata.org/entity/', 'wd:', st)
  return re.sub(r'http://ldf.fi/nbf/', 'nbf:', st)

applyPrefix('http://ldf.fi/nbf/p1869')

'nbf:p1869'

In [5]:
# construct a sparql query block with values (bs_id wiki_id)

#arr = ["({} {})".format(applyPrefix(ob.get('id')), applyPrefix(ob.get('wiki'))) for ob in res[:100] if ob.get('wiki')]
arr = ["(<{}> <{}>)".format(ob.get('id'), ob.get('wiki')) for ob in nbf2wiki if ob.get('wiki')]
print(len(arr))
VALUES = " ".join(arr)
VALUES#[:500]

6029


'(<http://ldf.fi/nbf/p1869> <http://www.wikidata.org/entity/Q4063683>) (<http://ldf.fi/nbf/p3876> <http://www.wikidata.org/entity/Q11865392>) (<http://ldf.fi/nbf/p4912> <http://www.wikidata.org/entity/Q3268888>) (<http://ldf.fi/nbf/p6241> <http://www.wikidata.org/entity/Q18760019>) (<http://ldf.fi/nbf/p249> <http://www.wikidata.org/entity/Q6224449>) (<http://ldf.fi/nbf/p2678> <http://www.wikidata.org/entity/Q11886278>) (<http://ldf.fi/nbf/p2415> <http://www.wikidata.org/entity/Q11886018>) (<http://ldf.fi/nbf/p1880> <http://www.wikidata.org/entity/Q5561382>) (<http://ldf.fi/nbf/p282> <http://www.wikidata.org/entity/Q233277>) (<http://ldf.fi/nbf/p2546> <http://www.wikidata.org/entity/Q297032>) (<http://ldf.fi/nbf/p2552> <http://www.wikidata.org/entity/Q17384404>) (<http://ldf.fi/nbf/p2762> <http://www.wikidata.org/entity/Q17381945>) (<http://ldf.fi/nbf/p5047> <http://www.wikidata.org/entity/Q6458065>) (<http://ldf.fi/nbf/p2417> <http://www.wikidata.org/entity/Q3362497>) (<http://ldf.fi/n

# Query artworks from Wikidata

In [6]:
sparql2 = SPARQLWrapper("https://query.wikidata.org/sparql")
# 
sparql2.setQuery(
""" PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX schema: <http://schema.org/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
prefix p: <http://www.wikidata.org/prop/> 
PREFIX bd: <http://www.bigdata.com/rdf#>
prefix wds: <http://www.wikidata.org/entity/statement/>
prefix ps: <http://www.wikidata.org/prop/statement/> 
prefix pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX people: <http://ldf.fi/schema/yoma/people/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX mwapi: <https://www.mediawiki.org/ontology#API/>
PREFIX nbf: <http://ldf.fi/nbf/>

SELECT DISTINCT ?bs ?wiki ?personLabel ?work ?workLabel ?workImage ?workModel ?workModelLabel ?europeana
WHERE {
  
  VALUES (?bs ?wiki) {"""+VALUES+""" }
  
  { ?wiki wdt:P800 ?work } # wdt=notable work
  UNION
  { ?work wdt:P1647*/wdt:P170 ?wiki } # wdt=subclass/creator
  OPTIONAL { ?work wdt:P18 ?workImage }
  
  OPTIONAL { ?work (wdt:P180|wdt:P2634) ?workModel . ?workModel wdt:P31 wd:Q5 }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,fi,sv". }

  OPTIONAL { ?wiki wdt:P7704 ?europeana }
} """)

sparql2.setReturnFormat(JSON)
sparql2.setMethod('POST')
results = sparql2.query().convert()

res = results["results"]["bindings"]
print(f"{len(res)} results")

print(f"Datafields {results['head']['vars']}")
res = convertDatatypes(results)

for ob in res[:10]:
  print(ob)

8842 results
Datafields ['bs', 'wiki', 'personLabel', 'work', 'workLabel', 'workImage', 'workModel', 'workModelLabel', 'europeana']
{'work': 'http://www.wikidata.org/entity/Q20782658', 'workModel': 'http://www.wikidata.org/entity/Q4411191', 'bs': 'http://ldf.fi/nbf/p5271', 'wiki': 'http://www.wikidata.org/entity/Q4411191', 'workLabel': 'Self-Portrait', 'workModelLabel': 'Sigrid Schauman', 'europeana': 'agent/base/42056'}
{'work': 'http://www.wikidata.org/entity/Q20781521', 'workModel': 'http://www.wikidata.org/entity/Q5940496', 'bs': 'http://ldf.fi/nbf/p4362', 'wiki': 'http://www.wikidata.org/entity/Q5940496', 'workLabel': 'Self-Portrait', 'workModelLabel': 'Aleksander Lauréus', 'workImage': 'http://commons.wikimedia.org/wiki/Special:FilePath/Alexander%20Laur%C3%A9us%20-%20Self-Portrait%20-%20A%20III%201893%20-%20Finnish%20National%20Gallery.jpg'}
{'work': 'http://www.wikidata.org/entity/Q20779048', 'workModel': 'http://www.wikidata.org/entity/Q14417101', 'bs': 'http://ldf.fi/nbf/p4873

In [7]:
pd.DataFrame(res)

,work,workModel,bs,wiki,workLabel,workModelLabel,europeana,workImage
0,http://www.wikidata.org/entity/Q20782658,http://www.wikidata.org/entity/Q4411191,http://ldf.fi/nbf/p5271,http://www.wikidata.org/entity/Q4411191,Self-Portrait,Sigrid Schauman,agent/base/42056,NaN
1,http://www.wikidata.org/entity/Q20781521,http://www.wikidata.org/entity/Q5940496,http://ldf.fi/nbf/p4362,http://www.wikidata.org/entity/Q5940496,Self-Portrait,Aleksander Lauréus,NaN,http://commons.wikimedia.org/wiki/Special:File...
2,http://www.wikidata.org/entity/Q20779048,http://www.wikidata.org/entity/Q14417101,http://ldf.fi/nbf/p4873,http://www.wikidata.org/entity/Q14417101,Self Portrait,Yrjö Ollila,NaN,http://commons.wikimedia.org/wiki/Special:File...
3,http://www.wikidata.org/entity/Q18599449,http://www.wikidata.org/entity/Q109294635,http://ldf.fi/nbf/p5854,http://www.wikidata.org/entity/Q11865103,Miss Brigita Johanna Lagerstam,Brita Johanna Lagerstam,NaN,http://commons.wikimedia.org/wiki/Special:File...
4,http://www.wikidata.org/entity/Q11891373,http://www.wikidata.org/entity/Q153159,http://ldf.fi/nbf/p4236,http://www.wikidata.org/entity/Q5911678,"Runeberg, Lönnrot, Snellman",Elias Lönnrot,NaN,NaN
...,...,...,...,...,...,...,...,...
8837,http://www.wikidata.org/entity/Q112151842,NaN,http://ldf.fi/nbf/p901,http://www.wikidata.org/entity/Q170068,Lake Ruovesi,NaN,agent/base/61358,http://commons.wikimedia.org/wiki/Special:File...
8838,http://www.wikidata.org/entity/Q112764089,NaN,http://ldf.fi/nbf/p2790,http://www.wikidata.org/entity/Q938282,Sunnuntaiaamu talonpoikaistuvassa,NaN,agent/base/14212,http://commons.wikimedia.org/wiki/Special:File...
8839,http://www.wikidata.org/entity/Q111801916,NaN,http://ldf.fi/nbf/p830,http://www.wikidata.org/entity/Q216904,Olviretki Schleusingenissa (TV Movie 1967),NaN,NaN,NaN
8840,http://www.wikidata.org/entity/Q110864858,NaN,http://ldf.fi/nbf/p910,http://www.wikidata.org/entity/Q356644,Summer evening,NaN,agent/base/147831,http://commons.wikimedia.org/wiki/Special:File...


# Query Europeana

## entries having europeana link in wikidata

In [8]:
sparql2 = SPARQLWrapper("https://query.wikidata.org/sparql")
# 
sparql2.setQuery(
""" PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX schema: <http://schema.org/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
prefix p: <http://www.wikidata.org/prop/> 
PREFIX bd: <http://www.bigdata.com/rdf#>
prefix wds: <http://www.wikidata.org/entity/statement/>
prefix ps: <http://www.wikidata.org/prop/statement/> 
prefix pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX people: <http://ldf.fi/schema/yoma/people/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX mwapi: <https://www.mediawiki.org/ontology#API/>
PREFIX nbf: <http://ldf.fi/nbf/>

SELECT DISTINCT ?wiki_id ?bs ?link
WHERE {
  
  ?wiki_id wdt:P7704 [] ; wdt:P9324 ?bs .
  
  { ?wiki_id wdt:P214 ?viaf . BIND(CONCAT('http://viaf.org/viaf/', STR(?viaf)) AS ?link) }
  UNION 
  { ?wiki_id wdt:P227 ?gnd . BIND(CONCAT('http://d-nb.info/gnd/', STR(?gnd)) AS ?link) }
  UNION 
  { ?wikipedia_fi schema:about ?wiki_id .
    FILTER (REGEX(STR(?wikipedia_fi),'fi.wikipedia.org'))
    BIND(REPLACE(STR(?wikipedia_fi), 'https://fi.wikipedia.org/wiki/', 'http://dbpedia.org/resource/') AS ?link)
  }
  UNION
  { ?wiki_id wdt:P7704 [].
      BIND(REPLACE(STR(?wiki_id), '/www.wikidata.org/entity/', '/wikidata.org/entity/') AS ?link)
  }
  UNION
  { ?wiki_id wdt:P7704 [].
      BIND(STR(?wiki_id) AS ?link)
  }
  
} LIMIT 3000
 """)

sparql2.setReturnFormat(JSON)
sparql2.setMethod('POST')
results = sparql2.query().convert()

res = results["results"]["bindings"]
print(f"{len(res)} results")

print(f"Datafields {results['head']['vars']}")
res = convertDatatypes(results)

for ob in res[:10]:
  print(ob)

986 results
Datafields ['wiki_id', 'bs', 'link']
{'wiki_id': 'http://www.wikidata.org/entity/Q5716240', 'bs': 'p677', 'link': 'http://viaf.org/viaf/71419574'}
{'wiki_id': 'http://www.wikidata.org/entity/Q5716240', 'bs': 'p677', 'link': 'http://d-nb.info/gnd/102455333'}
{'wiki_id': 'http://www.wikidata.org/entity/Q5716240', 'bs': 'p677', 'link': 'http://www.wikidata.org/entity/Q5716240'}
{'wiki_id': 'http://www.wikidata.org/entity/Q5716240', 'bs': 'p677', 'link': 'http://wikidata.org/entity/Q5716240'}
{'wiki_id': 'http://www.wikidata.org/entity/Q132723', 'bs': 'p355', 'link': 'http://viaf.org/viaf/147277'}
{'wiki_id': 'http://www.wikidata.org/entity/Q211542', 'bs': 'p4752', 'link': 'http://viaf.org/viaf/85458095'}
{'wiki_id': 'http://www.wikidata.org/entity/Q241432', 'bs': 'p5353', 'link': 'http://viaf.org/viaf/22116171'}
{'wiki_id': 'http://www.wikidata.org/entity/Q263080', 'bs': 'p2357', 'link': 'http://viaf.org/viaf/10651135'}
{'wiki_id': 'http://www.wikidata.org/entity/Q311037', 'bs

In [9]:
arr = {"(nbf:{} <{}>)".format(ob.get('bs'), ob.get('link')) for ob in res}
print(len(arr))

VALUES_B = " ".join(list(arr))
VALUES_B[:500]

986


'(nbf:p357 <http://www.wikidata.org/entity/Q193111>) (nbf:p3078 <http://www.wikidata.org/entity/Q55007624>) (nbf:p2141 <http://www.wikidata.org/entity/Q333297>) (nbf:p4522 <http://d-nb.info/gnd/1181455731>) (nbf:p6288 <http://www.wikidata.org/entity/Q4348368>) (nbf:p3184 <http://wikidata.org/entity/Q4115633>) (nbf:p2725 <http://d-nb.info/gnd/133629341>) (nbf:p4317 <http://d-nb.info/gnd/135023629>) (nbf:p455 <http://dbpedia.org/resource/Leevi_Madetoja>) (nbf:p3999 <http://wikidata.org/entity/Q2342'

## Fetch actor urls and related works of art from europeana

In [12]:
sparql3 = SPARQLWrapper("http://sparql.europeana.eu/")
# 
q = """ PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>
PREFIX ore: <http://www.openarchives.org/ore/terms/>

PREFIX nbf: <http://ldf.fi/nbf/>
PREFIX wd: <http://www.wikidata.org/entity/>

SELECT DISTINCT ?bs ?edm ?item
WHERE {
  VALUES (?bs ?wiki_id) { """+VALUES_B+""" }
  ?edm owl:sameAs? ?wiki_id .
  ?item dc:creator ?edm ; 
    edm:type ?type .
} LIMIT 20000 """
print(f'Query length {len(q)}')

sparql3.setQuery(q)


sparql3.setReturnFormat(JSON)
sparql3.setMethod('POST')
results = sparql3.query().convert()

res = convertDatatypes(results)
print(f"{len(res)} results")

print(f'Works by {len({ob.get("edm") for ob in res})} distinct artists')

for ob in res[:10]:
  print(ob)

Query length 50075
6389 results
Works by 73 distinct artists
{'bs': 'http://ldf.fi/nbf/p992', 'edm': 'http://dbpedia.org/resource/Jean_Sibelius', 'item': 'http://data.europeana.eu/proxy/europeana/03707/masc__URL_play_asp_ID_0240417'}
{'bs': 'http://ldf.fi/nbf/p992', 'edm': 'http://dbpedia.org/resource/Jean_Sibelius', 'item': 'http://data.europeana.eu/proxy/europeana/03707/masc__URL_play_asp_ID_0240423'}
{'bs': 'http://ldf.fi/nbf/p5353', 'edm': 'http://dbpedia.org/resource/Kaija_Saariaho', 'item': 'http://data.europeana.eu/proxy/europeana/03707/masc__URL_play_asp_ID_0834713'}
{'bs': 'http://ldf.fi/nbf/p992', 'edm': 'http://dbpedia.org/resource/Jean_Sibelius', 'item': 'http://data.europeana.eu/proxy/europeana/03707/masc__URL_play_asp_ID_0893964'}
{'bs': 'http://ldf.fi/nbf/p2796', 'edm': 'http://dbpedia.org/resource/Alfred_William_Finch', 'item': 'http://data.europeana.eu/proxy/europeana/04101/50176C934ABE41BA121E8B49E9467665FA9DBBAA'}
{'bs': 'http://ldf.fi/nbf/p5353', 'edm': 'http://dbpe

In [13]:
pd.DataFrame(res)

,bs,edm,item
0,http://ldf.fi/nbf/p992,http://dbpedia.org/resource/Jean_Sibelius,http://data.europeana.eu/proxy/europeana/03707...
1,http://ldf.fi/nbf/p992,http://dbpedia.org/resource/Jean_Sibelius,http://data.europeana.eu/proxy/europeana/03707...
2,http://ldf.fi/nbf/p5353,http://dbpedia.org/resource/Kaija_Saariaho,http://data.europeana.eu/proxy/europeana/03707...
3,http://ldf.fi/nbf/p992,http://dbpedia.org/resource/Jean_Sibelius,http://data.europeana.eu/proxy/europeana/03707...
4,http://ldf.fi/nbf/p2796,http://dbpedia.org/resource/Alfred_William_Finch,http://data.europeana.eu/proxy/europeana/04101...
...,...,...,...
6384,http://ldf.fi/nbf/p2623,http://d-nb.info/gnd/118567055,http://data.europeana.eu/proxy/provider/920038...
6385,http://ldf.fi/nbf/p2623,http://d-nb.info/gnd/118567055,http://data.europeana.eu/proxy/provider/920038...
6386,http://ldf.fi/nbf/p2623,http://d-nb.info/gnd/118567055,http://data.europeana.eu/proxy/provider/920038...
6387,http://ldf.fi/nbf/p2623,http://d-nb.info/gnd/118567055,http://data.europeana.eu/proxy/provider/920038...
